In [1]:
import pandas as pd
import numpy as np
import json
import shutil

In [2]:
dta = pd.read_csv('CFL_map_data_V1.csv', encoding='latin-1')

dta['date'] = pd.to_datetime(dta['mY'], format='%Y/%m')
dta.drop('mY', axis=1, inplace=True)
dta['week_avg'] = dta[['Montee_Weekday_vac', 'Descente_Weekday_vac']].sum(axis=1)

In [3]:
stats = dta.groupby(['Loc_map', dta.date.dt.year, dta.date.dt.month], observed=False).agg({'week_avg': 'sum'})
stats = stats.rename_axis(['stop', 'year', 'month'])

In [4]:
d = stats.reindex(pd.MultiIndex.from_product(stats.index.levels)).groupby(level=[0, 1]).agg(list)

In [5]:
def to_nested_dict(dta):
    res = {}
    for keys, val in dta.to_dict(orient='index').items():
        target = res
        for key in keys:
            target = target.setdefault(key, {})
        for k, v in val.items():
            target[k] = [None if pd.isnull(s) else int(s) for s in v]
    return res

In [7]:
with open('trainstats.json', 'w', encoding='utf-8') as f:
    json.dump(to_nested_dict(d), f)
shutil.copyfile('trainstats.json', '../../public/data/publictransport/trainstats.json')

'../../public/data/publictransport/trainstats.json'

In [9]:
dta

,Ligne new,Loc_map,Montee_Weekday_vac,Descente_Weekday_vac,date,week_avg
0,10,Clervaux,14002,13877,2017-01-01,27879
1,10,Colmar-Berg,4013,3837,2017-01-01,7850
2,10,Cruchten,2579,2494,2017-01-01,5073
3,10,Diekirch,19702,15527,2017-01-01,35229
4,10,Dommeldange,18129,31227,2017-01-01,49356
...,...,...,...,...,...,...
9020,90,Bettembourg,58298,66863,2023-01-01,125161
9021,90,Hettange-Grande,25330,19337,2023-01-01,44667
9022,90,Howald,10933,16571,2023-01-01,27504
9023,90,Luxembourg,262230,276114,2023-01-01,538344
